In [19]:
import pandas as pd
import numpy as np

In [20]:
df = pd.read_csv('../data/vancouver_real_estate.csv', index_col = 0)
df.reset_index(inplace=True)
df.drop(['level_0', 'index'], axis=1, inplace=True)
df


,address,zip_code,price,bed,bath,sqr_footage,property_link
0,4091 W 34th Ave,V6N 2L6,"$2,788,000",5 beds,2 baths,"1,653",https://www.redfin.com/bc/vancouver/4091-W-34t...
1,4067 W 37th Ave,V6N 2W6,"$4,800,000",6 beds,4 baths,"4,189",https://www.redfin.com/bc/vancouver/4067-W-37t...
2,4056 W 36th Ave,V6N 2S9,"$6,750,000",4 beds,3 baths,"3,837",https://www.redfin.com/bc/vancouver/4056-W-36t...
3,3968 W 23rd Ave,V6S 1L2,"$3,388,000",4 beds,3.5 baths,"2,359",https://www.redfin.com/bc/vancouver/3968-W-23r...
4,4022 W 30th Ave,V6S 1X5,"$6,590,000",5 beds,5.5 baths,"4,190",https://www.redfin.com/bc/vancouver/4022-W-30t...
...,...,...,...,...,...,...,...
11120,1931 East 7TH Ave #1927,V5N 1S3,"$3,598,000",7 beds,5 baths,"3,063",https://www.redfin.com/BC/Vancouver/1931-E-7th...
11121,3311 Sophia St,V5V 3T4,"$2,775,000",6 beds,5.5 baths,"3,073",https://www.redfin.com/bc/vancouver/3311-Sophi...
11122,3778 Wakefield C,V5G 2C4,"$2,190,000",4 beds,2.5 baths,"2,365",https://www.redfin.com/bc/burnaby/3778-Wakefie...
11123,2829 Ash St #101,V5Z 4P5,"$919,000",2 beds,2 baths,924,https://www.redfin.com/bc/vancouver/2829-Ash-S...


In [21]:

# Convert empty strings and None to NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# Convert price to float
df["price"] = df["price"].replace(r"[\$,]", "", regex=True).astype(float)

# Convert bed to integer (using Int64 which supports NaN)
df["bed"] = df["bed"].astype(str).str.extract(r"(\d+)").astype("Int64")

# Convert bath to float (supports half baths)
df["bath"] = df["bath"].astype(str).str.extract(r"(\d+\.?\d*)").astype(float)

# Convert sqr_footage to float, handling missing values properly
df["sqr_footage"] = (
    df["sqr_footage"]
    .astype(str)
    .str.replace(",", "", regex=True)  
)



In [23]:
df.drop_duplicates()

,address,zip_code,price,bed,bath,sqr_footage,property_link
0,4091 W 34th Ave,V6N 2L6,2788000.0,5,2.0,1653,https://www.redfin.com/bc/vancouver/4091-W-34t...
1,4067 W 37th Ave,V6N 2W6,4800000.0,6,4.0,4189,https://www.redfin.com/bc/vancouver/4067-W-37t...
2,4056 W 36th Ave,V6N 2S9,6750000.0,4,3.0,3837,https://www.redfin.com/bc/vancouver/4056-W-36t...
3,3968 W 23rd Ave,V6S 1L2,3388000.0,4,3.5,2359,https://www.redfin.com/bc/vancouver/3968-W-23r...
4,4022 W 30th Ave,V6S 1X5,6590000.0,5,5.5,4190,https://www.redfin.com/bc/vancouver/4022-W-30t...
...,...,...,...,...,...,...,...
10860,63 Keefer Pl #1510,V6B 6N6,529900.0,1,1.0,579,https://www.redfin.com/bc/vancouver/63-Keefer-...
10861,188 Keefer St #816,V6A 0E3,599000.0,1,1.0,608,https://www.redfin.com/bc/vancouver/188-Keefer...
10862,406 UNION St,V6A 2B6,2599000.0,<NA>,NaN,6100,https://www.redfin.com/bc/vancouver/406-Union-...
10863,511 E Georgia St,V6A 1Z8,1299000.0,<NA>,NaN,—,https://www.redfin.com/bc/vancouver/511-E-Geor...
